### Import and Setup

In [ ]:
!pip install unidecode

     |████████████████████████████████| 235 kB 4.2 MB/s 


In [ ]:
import json
import re
from unidecode import unidecode
from nltk.tokenize import word_tokenize, sent_tokenize
import nltk
nltk.download('punkt')
from collections import Counter

from google.colab import drive
drive.mount('/content/drive')

[nltk_data] Downloading package punkt to /root/nltk_data...
[nltk_data]   Unzipping tokenizers/punkt.zip.
Mounted at /content/drive


In [ ]:
import torch
import torch.nn as nn
import torch.nn.functional as F
import numpy as np

device = "cuda" if torch.cuda.is_available() else "cpu"
assert device == "cuda"  

seed = 42
np.random.seed(seed)
torch.manual_seed(seed)
torch.cuda.manual_seed(seed)

### Data

In [ ]:
f = open('/content/drive/Shareddrives/MIT NLP 8.864/Data/drake.json')
drake = json.load(f)
f.close()

f = open('/content/drive/Shareddrives/MIT NLP 8.864/Data/tswift.json')
taylor = json.load(f)
f.close()

drake = [drake['songs'][i]['lyrics'] for i in range(len(drake['songs']))]
taylor = [taylor['songs'][i]['lyrics'] for i in range(len(taylor['songs']))]

taylor_lyrics = [re.sub('\u2005', ' ', re.sub(r'[\(\[].*?[\)\]]', '', taylor[i])).split('\n') for i in range(len(taylor))]
taylor_lyrics = [[unidecode(i) for i in taylor_lyrics[j]] for j in range(len(taylor_lyrics))]
taylor_lyrics = [[re.sub('\d+EmbedShare URLCopyEmbedCopy', '', i) for i in taylor_lyrics[j]] for j in range(len(taylor_lyrics))]
taylor_lyrics = [[re.sub('\d+.EmbedShare URLCopyEmbedCopy', '', i) for i in taylor_lyrics[j]] for j in range(len(taylor_lyrics))]
taylor_lyrics = [[re.sub('EmbedShare URLCopyEmbedCopy', '', i) for i in taylor_lyrics[j]] for j in range(len(taylor_lyrics))]
taylor_lyrics = [[i for i in taylor_lyrics[j] if i != ''] for j in range(len(taylor_lyrics))]

drake_lyrics = [re.sub('\u2005', ' ', re.sub(r'[\(\[].*?[\)\]]', '', drake[i])).split('\n') for i in range(len(drake))]
drake_lyrics = [[unidecode(i) for i in drake_lyrics[j]] for j in range(len(drake_lyrics))]
drake_lyrics = [[re.sub('\d+EmbedShare URLCopyEmbedCopy', '', i) for i in drake_lyrics[j]] for j in range(len(drake_lyrics))]
drake_lyrics = [[re.sub('\d+.EmbedShare URLCopyEmbedCopy', '', i) for i in drake_lyrics[j]] for j in range(len(drake_lyrics))]
drake_lyrics = [[re.sub('EmbedShare URLCopyEmbedCopy', '', i) for i in drake_lyrics[j]] for j in range(len(drake_lyrics))]
drake_lyrics = [[i for i in drake_lyrics[j] if i != ''] for j in range(len(drake_lyrics))]

taylor_lyrics = [[line1 + ', ' + line2 for line1,line2 in zip(song[0::2], song[1::2])] for song in taylor_lyrics]
drake_lyrics = [[line1 + ', ' + line2 for line1,line2 in zip(song[0::2], song[1::2])] for song in drake_lyrics]

drake_tokenized = [[word_tokenize(drake_lyrics[i][j]) for j in range(len(drake_lyrics[i]))] for i in range(len(drake_lyrics))]
taylor_tokenized = [[word_tokenize(taylor_lyrics[i][j]) for j in range(len(taylor_lyrics[i]))] for i in range(len(taylor_lyrics))]

drake_tokenized = [[[word.lower() for word in line] for line in song] for song in drake_tokenized]
taylor_tokenized = [[[word.lower() for word in line] for line in song] for song in taylor_tokenized]

drake_length = sum([[len(sent) for sent in song] for song in drake_tokenized], [])
taylor_length = sum([[len(sent) for sent in song] for song in taylor_tokenized], [])

drake_lyrics = sum([[sent for sent in song if (len(sent) >= 10 and len (sent) <= 30)] for song in drake_tokenized], [])
taylor_lyrics = sum([[sent for sent in song if (len(sent) >= 10 and len (sent) <= 30)] for song in taylor_tokenized], [])

taylor_vocab = sum(taylor_lyrics,[])
drake_vocab = sum(drake_lyrics,[])

def unique(list1):
     
    # insert the list to the set
    list_set = set(list1)
    # convert the set to the list
    unique_list = (list(list_set))
    return unique_list

vocab = taylor_vocab + drake_vocab
vocab_counts = Counter(vocab)
vocab = unique(vocab)
vocab = ['<pad>','<unk>','<s>', '</s>'] + vocab

from torch.utils import data
import torch

# These IDs are reserved.
MAX_SENT_LENGTH = 30
MAX_SENT_LENGTH_PLUS_SOS_EOS = 32
PAD_INDEX = 0
UNK_INDEX = 1
SOS_INDEX = 2
EOS_INDEX = 3
RARE_WORD_TRESHOLD = 0

vocab_counts['<pad>'] = RARE_WORD_TRESHOLD + 1
vocab_counts['<unk>'] = RARE_WORD_TRESHOLD + 1
vocab_counts['<s>'] = RARE_WORD_TRESHOLD + 1
vocab_counts['</s>'] = RARE_WORD_TRESHOLD + 1

class TSTDataset(data.Dataset):
    def __init__(self, taylor_sentences, drake_sentences, vocab, vocab_counts, sampling=1.):
        self.taylor_sentences = taylor_sentences[:int(len(taylor_sentences) * sampling)]
        self.drake_sentences = drake_sentences[:int(len(drake_sentences) * sampling)]

        self.max_seq_length = MAX_SENT_LENGTH_PLUS_SOS_EOS
        self.vocab = vocab
        self.vocab_counts = vocab_counts

        self.v2id = {v : i for i, v in enumerate(self.vocab)}
        self.id2v = {val : key for key, val in self.v2id.items()}
    
    def __len__(self):
        return min(len(self.taylor_sentences), len(self.drake_sentences))
    
    def __getitem__(self, index):
        taylor_sent = self.taylor_sentences[index]
        taylor_len = len(taylor_sent) + 2   # add <s> and </s> to each sentence
        taylor_id = []
        for w in taylor_sent:
            if w not in self.vocab:
                w = '<unk>'
            if vocab_counts[w] <= RARE_WORD_TRESHOLD:
                w = '<unk>'
            taylor_id.append(self.v2id[w])

        taylor_id = ([SOS_INDEX] + taylor_id + [EOS_INDEX] + [PAD_INDEX] *
                  (self.max_seq_length - taylor_len))

        drake_sent = self.drake_sentences[index]
        drake_len = len(drake_sent) + 2   # add <s> and </s> to each sentence
        drake_id = []
        for w in drake_sent:
            if w not in self.vocab:
                w = '<unk>'
            if vocab_counts[w] <= RARE_WORD_TRESHOLD:
                w = '<unk>'
            drake_id.append(self.v2id[w])

        drake_id = ([SOS_INDEX] + drake_id + [EOS_INDEX] + [PAD_INDEX] *
                  (self.max_seq_length - drake_len))

        return torch.tensor(taylor_id), taylor_len, torch.tensor(drake_id), drake_len

dataset = TSTDataset(taylor_lyrics, drake_lyrics, vocab, vocab_counts)

In [ ]:
test_pct = 0.2
valid_pct = 0.1

train_dataset, test_dataset = torch.utils.data.random_split(dataset, [int(len(dataset)*(1-test_pct)),len(dataset)-int(len(dataset)*(1-test_pct))])
valid_dataset, train_dataset = torch.utils.data.random_split(train_dataset, [int(len(dataset)*valid_pct),len(train_dataset)-int(len(dataset)*valid_pct)])

### Encoder

In [ ]:
from torch.nn.utils.rnn import pack_padded_sequence, pad_packed_sequence

class Encoder(nn.Module):
  def __init__(self, input_size, hidden_size, dropout=0.):
    """
    Inputs: 
      - `input_size`: an int representing the RNN input size.
      - `hidden_size`: an int representing the RNN hidden size.
      - `dropout`: a float representing the dropout rate during training. Note
          that for 1-layer RNN this has no effect since dropout only applies to
          outputs of intermediate layers.
    """
    super(Encoder, self).__init__()
    self.rnn = nn.GRU(input_size, hidden_size, num_layers=1, batch_first=True,
                      dropout=dropout, bidirectional=False)

  def forward(self, inputs, lengths, init_state=None):
    """
    Inputs:
      - `inputs`: a 3d-tensor of shape (batch_size, max_seq_length, embed_size)
          representing a batch of padded embedded word vectors of source
          sentences.
      - `lengths`: a 1d-tensor of shape (batch_size,) representing the sequence
          lengths of `inputs`.

    Returns:
      - `outputs`: a 3d-tensor of shape
        (batch_size, max_seq_length, hidden_size).
      - `finals`: a 3d-tensor of shape (num_layers, batch_size, hidden_size).
      Hint: `outputs` and `finals` are both standard GRU outputs. Check:
      https://pytorch.org/docs/stable/nn.html#gru
    """
    # Our variable-length inputs are padded to the same length for batching
    # Here we "pack" them for computational efficiency (see note below)
    packed = pack_padded_sequence(inputs, lengths.cpu(), batch_first=True,
                                  enforce_sorted=False)
    outputs, finals = self.rnn(packed, init_state)
    outputs, _ = pad_packed_sequence(outputs, batch_first=True,
                                     total_length=32)
    return outputs, finals

### Decoder

In [ ]:
class GeneratorTransferredSampled(nn.Module):
  """Define standard linear + softmax generation step."""
  def __init__(self, hidden_size, vocab_size, src_embed, gamma=0.1):
    """
    Inputs:
      - `src_embed`: a 2d-tensor of shape (vocab_size, embed_size )
    """
    super(GeneratorTransferredSampled, self).__init__()
    self.proj = nn.Linear(hidden_size, vocab_size, bias=True)
    self.gamma = gamma
    self.softmax = nn.Softmax(dim = 2)
    self.src_embed = src_embed

  def embedding(self,x):
    return torch.matmul(x,self.src_embed.weight)
    
  def gumbel_softmax(self,logits, eps=1e-20):
    U = torch.rand(logits.shape).to(device)
    G = -torch.log(-torch.log(U + eps) + eps).to(device)
    return self.softmax((logits + G) / self.gamma)

  def forward(self, x):
    logits = self.proj(x)
    prob = self.gumbel_softmax(logits)
    output = self.embedding(prob)
    word  = logits.argmax(dim = 2, keepdim = False)
    return output, prob, word

In [ ]:
class Decoder(nn.Module):
  """An RNN decoder + generator with GRU"""

  def __init__(self, input_size, hidden_size, max_len,generator, num_layers = 1, dropout=0.):
    """
      Inputs:
        - `input_size` , `hidden_size`, and `dropout` the same as in Encoder.
    """
    super(Decoder, self).__init__()
    self.input_size = input_size
    self.hidden_size = hidden_size
    self.rnn = nn.GRU(input_size, hidden_size, num_layers=num_layers, batch_first=True,
                      dropout=dropout, bidirectional=False)
    self.generator = generator
    self.max_len = max_len

  def forward_step(self, prev_embed, hidden):
    """Helper function for forward below:
       Perform a single decoder step (1 word).

       Inputs:
      - `prev_embed`: a 3d-tensor of shape (batch_size, 1, embed_size = vocab_size)
          representing the padded embedded word vectors at this step in training
      - `hidden`: a 3d-tensor of shape (1, batch_size, hidden_size) representing
          the current hidden state.

      Returns:
      - `hidden`: a 3d-tensor of shape (1, batch_size, hidden_size)
          representing the current decoder hidden state.
      - `output`: a 3d-tensor of shape (batch_size, max_len, vocab_size)
          representing the total generated outputs.
    """

    pre_output,hidden = self.rnn(prev_embed,hidden)
    return hidden, pre_output

  def forward(self, input, encoder_finals,max_len, hidden=None):
    """Unroll the decoder one step at a time.

    Inputs:
      - `inputs`: a 3d-tensor of shape (batch_size, 1, embed_size)
          representing a batch of padded embedded word vectors of SOS . 
          If size is (batch_size,max_len, embed_size), then it is teacher forcing.
      - `encoder_finals`: a 3d-tensor of shape
          (num_enc_layers, batch_size, hidden_size) representing the final
          encoder hidden states used to initialize the initial decoder hidden
          states.
      - `hidden`: a 3d-tensor of shape (1, batch_size, hidden_size) representing
          the value to be used to initialize the initial decoder hidden states.
          If None, then use `encoder_finals`.
      - `max_len`: an int representing the maximum decoding length.
      - `style`: TAYLOR_STYLE or DRAKE_STYLE

    Returns:
      - `hidden`: a 3d-tensor of shape
          (num_layers, batch_size, hidden_size) representing the final hidden
          state for each element in the batch.
      - `outputs`: a 3d-tensor of shape
          (batch_size, max_len, hidden_size) representing the raw decoder
          outputs (before mapping to a `trg_vocab_size`-dim vector).
      - `logits_vectors`: a 3d-tensor of shape
          (batch_size, max_len, trg_vocab_size) representing the mapped decoder
          outputs.
      - `words`: a 3d-tensor of shape
          (batch_size, max_len, 1) representing output sentence and
          the corresponding word index (can be used for embedding)  
    """

    # Initialize decoder hidden state.
    if hidden is None:
      hidden = self.init_hidden(encoder_finals)
    output_vectors = []
    logits_vectors = []
    words = []
    for i in range(max_len) :
      hidden, prev_output = self.forward_step(input,hidden)
      input, logits, output_word = self.generator(prev_output)

      # input = torch.concat([input,torch.full(input.shape,style)], axis = -1)
      logits_vectors.append(logits)
      output_vectors.append(input)
      words.append(output_word)
    outputs = torch.cat(output_vectors, dim =1)
    logits_vectors = torch.cat(logits_vectors,dim = 1)
    words = torch.cat(words, axis = -1)
    return hidden, outputs , logits_vectors, words

  def forward_teacher(self, input, encoder_finals,max_len=None, hidden=None):
    """Unroll the decoder one step at a time.

    Inputs:
      - `inputs`: a 3d-tensor of shape (batch_size,max_len, embed_size)
          representing a batch of padded embedded word vectors of original 
          sentence and acts as  teacher forcing.

    Returns:
      - `hidden`: a 3d-tensor of shape
          (num_layers, batch_size, hidden_size) representing the final hidden
          state for each element in the batch.
      - `outputs`: a 3d-tensor of shape
          (batch_size, max_len, hidden_size) representing the raw decoder
          outputs (before mapping to a `trg_vocab_size`-dim vector).
      - `logits_vectors`: a 3d-tensor of shape
          (batch_size, max_len, trg_vocab_size) representing the mapped decoder
          outputs each represents the probability? 
      - `words`: a 3d-tensor of shape
          (batch_size, max_len, 1) representing output sentence and
          the corresponding word index (can be used for embedding)      
    """

    # Initialize decoder hidden state.
    if max_len is None:
      max_len = input.shape[1]
    if hidden is None:
      hidden = self.init_hidden(encoder_finals)
    output_vectors = []
    logits_vectors = []
    words = []
    for i in range(max_len) :
      hidden, prev_output = self.forward_step(input[:,i:i+1,:],hidden)
      output, logits , output_word= self.generator(prev_output)
      logits_vectors.append(logits)
      output_vectors.append(output)
      words.append(output_word)
    outputs = torch.cat(output_vectors, dim =1)
    logits_vectors = torch.cat(logits_vectors,dim = 1)
    words = torch.cat(words, axis = -1)
    return hidden, outputs ,logits_vectors, words

  def init_hidden(self, encoder_finals):
    """Use encoder final hidden state to initialize decoder's first hidden
       state.

       Input: `encoder_finals` is same as in forward()

       Returns: 
         - `decoder_init_hiddens`: a 3d-tensor of shape 
              (num_layers, batch_size, hidden_size) representing the initial
              hidden state of the decoder for each element in the batch 
    """
    decoder_init_hiddens=encoder_finals
    return decoder_init_hiddens

### Classifier

In [ ]:
class LSTMClassifier(nn.Module):
  def __init__(self, embed_size, hidden_size, vocab_size, LSTMlayers=1, dropout = 0.2):
    super(LSTMClassifier, self).__init__()
    self.embed = nn.Linear(vocab_size, embed_size)
    self.lstm = nn.LSTM(input_size=embed_size, hidden_size=hidden_size, num_layers=LSTMlayers, batch_first=True)
    self.dropout = nn.Dropout(dropout)
    self.fc1 =  nn.Linear(in_features=hidden_size, out_features=hidden_size+1)
    self.fc2 = nn.Linear(hidden_size+1, 1)

  def forward(self, seq):
    seq = self.embed(seq)
    output, (hidden,cell) = self.lstm(seq)
    output = self.dropout(output)
    output = self.fc1(output[:,-1])
    output = F.relu(output)
    output = self.fc2(output)
    return output
    return output.reshape(output.size(1), -1)

### Training

In [ ]:
### Work in progress
class TSTModel(nn.Module):
  def __init__(self, max_len, vocab_size, embed_size, hidden_size_z, hidden_size_y, line_embed, encoder, generator, decoder, classifier):
    super(TSTModel, self).__init__()

    self.hidden_size = hidden_size_y + hidden_size_z

    self.encoder = encoder
    self.generator = generator
    self.decoder = decoder
    self.classifier = classifier

    self.line_embed = line_embed
    self.y_embed_enc = nn.Embedding(2,hidden_size_y)
    self.y_embed_gen = nn.Embedding(2,hidden_size_y)

    self.max_len = max_len
    self.vocab_size = vocab_size
    self.embed_size = embed_size
    self.hidden_size_z = hidden_size_z
    self.hidden_size_y = hidden_size_y

  def forward(self, lines, line_lens, labels):
    encoded_lines = self.encode(lines, line_lens, labels)
    z = encoded_lines[1][-1][:,self.hidden_size_y:]

    h0_orig = torch.cat((self.y_embed_gen(labels),z), 1)[None,:]
    h0_tsf = torch.cat((self.y_embed_gen(1-labels),z), 1)[None,:]

    # Decode back into original form for reconstruction
    rec_orig = self.reconstruct(h0_orig, lines)
    # Decode into original and transferred forms for classification
    decode_orig = self.decode(h0_orig)
    decode_tsf = self.decode(h0_tsf)

    classifier_lines = torch.cat((decode_orig[2], decode_tsf[2], F.one_hot(lines, self.vocab_size).to(torch.float)), 0)
    pred_class = self.classifier(classifier_lines)

    return rec_orig, pred_class, decode_orig, decode_tsf

  def encode(self, lines, line_lens, labels):
    init_state = torch.cat((self.y_embed_enc(labels), torch.zeros((len(lines),self.hidden_size_z), device=device)), 1)[None,:].to(device)
    return self.encoder(self.line_embed(lines), line_lens, init_state)

  def reconstruct(self, h0, lines):
    original = self.line_embed(lines)
    return self.decoder.forward_teacher(original,h0)

  def decode(self, h0):
    target = self.line_embed(torch.tensor([SOS_INDEX]).repeat(h0.size()[1],1).to(device))
    return self.decoder.forward(target,h0,self.max_len)

In [ ]:
epochs = 1
class_epochs = 0
lr = 1e-3
batch_size = 32
print_every = 100

max_len = dataset.max_seq_length
vocab_size = len(vocab)
embed_size = 256
hidden_size_z = 256
hidden_size_y = 128
hidden_size = hidden_size_z + hidden_size_y
dropout = 0.2

TAYLOR_STYLE=1 # for information only, don't change
DRAKE_STYLE=0  # for information only, don't change
train_loader = data.DataLoader(train_dataset, batch_size=batch_size, shuffle=True)
valid_loader = data.DataLoader(valid_dataset, batch_size=batch_size, shuffle=True)
test_loader = data.DataLoader(test_dataset, batch_size=batch_size, shuffle=True)

line_embed = nn.Embedding(vocab_size, embed_size)
encoder = Encoder(embed_size,hidden_size)
generator = GeneratorTransferredSampled(hidden_size,vocab_size, line_embed)
decoder = Decoder(embed_size, hidden_size, max_len=vocab_size, generator = generator,dropout=dropout)
classifier = LSTMClassifier(embed_size, hidden_size_z, vocab_size, dropout =0.2)

# class_loss = nn.BCEWithLogitsLoss()
# optim_class = torch.optim.Adam(classifier.parameters(), lr=lr)
# classifier = classifier.to(device)
# classifier.train()
# for epoch in range(class_epochs):
#   epoch_class_loss = 0
#   for i,(taylor_lines, taylor_len, drake_lines, drake_len) in enumerate(train_loader):
#     lines = torch.cat((taylor_lines, drake_lines), 0).to(device)    
#     line_lens = torch.cat((taylor_len, drake_len), 0).to(device)
#     labels = torch.cat((torch.ones(size=(len(taylor_lines),), dtype=torch.int32),torch.zeros(size=(len(drake_lines),),dtype=torch.int32))).unsqueeze(1).to(device)
    
#     pred_class = classifier(F.one_hot(lines, vocab_size).to(torch.float))
#     loss_class = class_loss(input=pred_class, target=labels.to(torch.float))
#     epoch_class_loss += loss_class

#     optim_class.zero_grad()
#     loss_class.backward()
#     optim_class.step()
#   print(epoch_class_loss)

model = TSTModel(dataset.max_seq_length, vocab_size, embed_size, hidden_size_z, hidden_size_y, line_embed, encoder, generator, decoder, classifier).to(device)
optimizer = torch.optim.Adam(model.parameters(), lr=lr) 
rec_loss = nn.CrossEntropyLoss(reduction="mean",ignore_index = PAD_INDEX)
class_loss = nn.BCEWithLogitsLoss()

epoch_losses = []
for epoch in range(epochs):
  epoch_loss = 0
  epoch_class_loss = 0
  epoch_rec_loss = 0
  model.train()
  for i,(taylor_lines, taylor_len, drake_lines, drake_len) in enumerate(train_loader):
    lines = torch.cat((taylor_lines, drake_lines), 0).to(device)    
    line_lens = torch.cat((taylor_len, drake_len), 0).to(device)
    labels = torch.cat((torch.ones(size=(len(taylor_lines),), dtype=torch.int32),torch.zeros(size=(len(drake_lines),),dtype=torch.int32))).to(device)
    classifier_labels = torch.cat((labels,1-labels, labels)).unsqueeze(1)

    rec_orig, pred_class, decode_orig, decode_tsf = model(lines, line_lens, labels)

    loss_class = class_loss(pred_class, classifier_labels.to(torch.float))
    loss_rec = rec_loss(input=rec_orig[-2].permute(0,2,1), target=lines)

    # print(rec_orig[-2].contiguous().view(-1, rec_orig[-2].size(-1)).size())
    # print(lines.contiguous().view(-1).size())
    # print(rec_loss_sum(input=rec_orig[-2].permute(0,2,1), target=lines) / rec_loss(input=rec_orig[-2].permute(0,2,1), target=lines))
  
    loss = loss_class + loss_rec
    
    epoch_loss += loss
    epoch_class_loss += loss_class
    epoch_rec_loss += loss_rec

    optimizer.zero_grad()
    loss.backward()
    optimizer.step()
    
    if model.training and i % print_every == 0:
      print("Epoch Step: %d Loss: %f" % (i, loss / lines.size(0)))

  epoch_losses.append(epoch_loss)
  print("Finished Epoch ", epoch)
  print("Epoch Loss", epoch_loss.item())
  print("Classification Loss", epoch_class_loss.item())
  print("Reconstruction Loss", epoch_rec_loss.item())

/usr/local/lib/python3.7/dist-packages/torch/nn/modules/rnn.py:65: UserWarning: dropout option adds dropout after all but last recurrent layer, so non-zero dropout expects num_layers greater than 1, but got dropout=0.2 and num_layers=1
  "num_layers={}".format(dropout, num_layers))


Epoch Step: 0 Loss: 0.158666
Epoch Step: 100 Loss: 0.150101
Epoch Step: 200 Loss: 0.149192
Finished Epoch  0
Epoch Loss 2080.6787109375
Classification Loss 68.83372497558594
Reconstruction Loss 2011.8458251953125


In [ ]:
print("Epoch Loss", epoch_loss.item())

Epoch Loss 2080.6787109375


In [ ]:
correct_pred = 0
correct_pred_all = 0
model.eval()
with torch.no_grad():
  for i,(taylor_lines, taylor_len, drake_lines, drake_len) in enumerate(valid_loader):
    lines = torch.cat((taylor_lines, drake_lines), 0).to(device)    
    line_lens = torch.cat((taylor_len, drake_len), 0).to(device)
    labels = torch.cat((torch.ones(size=(len(taylor_lines),), dtype=torch.int32),torch.zeros(size=(len(drake_lines),),dtype=torch.int32))).to(device)
    classifier_labels = torch.cat((labels,1-labels, labels)).unsqueeze(1)

    rec_orig,pred_class, decode_orig,decode_tsf =  model(lines, line_lens, labels)
    correct_pred += torch.sum(1*(1*(torch.sigmoid(pred_class[-len(lines):])>=0.5) == labels.unsqueeze(1)))
    correct_pred_all += torch.sum(1*(1*(torch.sigmoid(pred_class)>=0.5) == classifier_labels))
  print(correct_pred / (2.*len(valid_dataset)))
  print(correct_pred_all / (3.*2.*len(valid_dataset)))

tensor(0.8514, device='cuda:0')
tensor(0.9505, device='cuda:0')


In [ ]:
with torch.no_grad():
  for i,(taylor_lines, taylor_len, drake_lines, drake_len) in enumerate(valid_loader):
    lines = torch.cat((taylor_lines, drake_lines), 0).to(device)    
    line_lens = torch.cat((taylor_len, drake_len), 0).to(device)
    labels = torch.cat((torch.ones(size=(len(taylor_lines),), dtype=torch.int32),torch.zeros(size=(len(drake_lines),),dtype=torch.int32))).to(device)
    classifier_labels = torch.cat((labels,1-labels, labels)).unsqueeze(1)

    rec_orig,pred_class,decode_orig,decode_tsf =  model(lines, line_lens, labels)
    
    break

In [ ]:
def lookup_words(x, vocab):
  return [vocab[i] for i in x]

print(lookup_words(decode_orig[3][0], vocab))
print(lookup_words(decode_tsf[3][0], vocab))
print(lookup_words(rec_orig[3][0], vocab))

['<s>', '<s>', 'cash-only', 'cash-only', 'cash-only', 'cash-only', 'cash-only', 'cash-only', 'cash-only', 'cash-only', 'cash-only', 'cash-only', 'cash-only', 'cash-only', 'cash-only', 'cash-only', 'cash-only', 'cash-only', 'cash-only', 'cash-only', 'cash-only', 'cash-only', 'cash-only', 'cash-only', 'cash-only', 'cash-only', 'cash-only', 'cash-only', 'cash-only', 'cash-only', 'cash-only', 'cash-only']
['<s>', 'it', 'it', 'it', 'it', 'it', 'it', 'it', 'it', 'it', 'it', 'it', 'it', 'it', 'it', 'it', 'it', 'it', 'it', 'it', 'it', 'it', 'it', 'it', 'it', 'it', 'it', 'it', 'it', 'it', 'it', 'it']
['<s>', '<s>', 'cash-only', 'cash-only', 'cash-only', 'cash-only', 'cash-only', 'cash-only', 'cash-only', 'cash-only', 'cash-only', 'cash-only', 'cash-only', 'cash-only', 'cash-only', 'cash-only', 'cash-only', 'cash-only', 'cash-only', 'cash-only', 'cash-only', 'cash-only', 'cash-only', 'cash-only', 'cash-only', 'cash-only', 'cash-only', 'cash-only', 'cash-only', 'cash-only', 'cash-only', 'cash-onl

In [ ]:
def greedy_decode(model, src_ids, src_lengths, max_len):
  """Greedily decode a sentence for EncoderDecoder."""

  with torch.no_grad():
    _, encoder_finals = model.encode(src_ids, src_lengths)
    prev_y = torch.ones(1, 1).fill_(SOS_INDEX).type_as(src_ids)

  output = []
  hidden = None

  for i in range(max_len):
    with torch.no_grad():
      hidden, outputs = model.decode(encoder_finals, prev_y, hidden)
      prob = model.generator(outputs[:, -1])

    _, next_word = torch.max(prob, dim=1)
    next_word = next_word.data.item()
    output.append(next_word)
    prev_y = torch.ones(1, 1).type_as(src_ids).fill_(next_word)

  output = np.array(output)

  # Cut off everything starting from </s>.
  first_eos = np.where(output == EOS_INDEX)[0]
  if len(first_eos) > 0:
    output = output[:first_eos[0]]
  return output

def lookup_words(x, vocab):
  return [vocab[i] for i in x]

In [ ]:
import nltk
nltk.download('punkt')
from nltk import word_tokenize
from nltk.translate.bleu_score import sentence_bleu
import numpy as np
import os

[nltk_data] Downloading package punkt to /root/nltk_data...
[nltk_data]   Unzipping tokenizers/punkt.zip.


In [6]:
!wget http://nlp.stanford.edu/data/glove.6B.zip
!unzip glove*.zip

print('Indexing word vectors.')
import numpy as np

embeddings_index = {}
f = open('glove.6B.100d.txt', encoding='utf-8')
for line in f:
    values = line.split()
    word = values[0]
    coefs = np.asarray(values[1:], dtype='float32')
    embeddings_index[word] = coefs
# f.close()

print('Found %s word vectors.' % len(embeddings_index))   

--2021-11-19 22:02:34--  http://nlp.stanford.edu/data/glove.6B.zip
Resolving nlp.stanford.edu (nlp.stanford.edu)... 171.64.67.140
Connecting to nlp.stanford.edu (nlp.stanford.edu)|171.64.67.140|:80... connected.
HTTP request sent, awaiting response... 302 Found
Location: https://nlp.stanford.edu/data/glove.6B.zip [following]
--2021-11-19 22:02:34--  https://nlp.stanford.edu/data/glove.6B.zip
Connecting to nlp.stanford.edu (nlp.stanford.edu)|171.64.67.140|:443... connected.
HTTP request sent, awaiting response... 301 Moved Permanently
Location: http://downloads.cs.stanford.edu/nlp/data/glove.6B.zip [following]
--2021-11-19 22:02:35--  http://downloads.cs.stanford.edu/nlp/data/glove.6B.zip
Resolving downloads.cs.stanford.edu (downloads.cs.stanford.edu)... 171.64.64.22
Connecting to downloads.cs.stanford.edu (downloads.cs.stanford.edu)|171.64.64.22|:80... connected.
HTTP request sent, awaiting response... 200 OK
Length: 862182613 (822M) [application/zip]
Saving to: ‘glove.6B.zip.5’

glove

In [ ]:
!wget http://nlp.stanford.edu/data/glove.6B.zip
!unzip -q glove.6B.zip

path_to_glove_file = os.path.join(
    os.path.expanduser("~"), ".keras/datasets/glove.6B.100d.txt"
)

embeddings_index = {}
with open(path_to_glove_file) as f:
    for line in f:
        word, coefs = line.split(maxsplit=1)
        coefs = np.fromstring(coefs, "f", sep=" ")
        embeddings_index[word] = coefs

print("Found %s word vectors." % len(embeddings_index))

--2021-11-19 16:11:11--  http://nlp.stanford.edu/data/glove.6B.zip
Resolving nlp.stanford.edu (nlp.stanford.edu)... 171.64.67.140
Connecting to nlp.stanford.edu (nlp.stanford.edu)|171.64.67.140|:80... connected.
HTTP request sent, awaiting response... 302 Found
Location: https://nlp.stanford.edu/data/glove.6B.zip [following]
--2021-11-19 16:11:11--  https://nlp.stanford.edu/data/glove.6B.zip
Connecting to nlp.stanford.edu (nlp.stanford.edu)|171.64.67.140|:443... connected.
HTTP request sent, awaiting response... 301 Moved Permanently
Location: http://downloads.cs.stanford.edu/nlp/data/glove.6B.zip [following]
--2021-11-19 16:11:12--  http://downloads.cs.stanford.edu/nlp/data/glove.6B.zip
Resolving downloads.cs.stanford.edu (downloads.cs.stanford.edu)... 171.64.64.22
Connecting to downloads.cs.stanford.edu (downloads.cs.stanford.edu)|171.64.64.22|:80... connected.
HTTP request sent, awaiting response... 200 OK
Length: 862182613 (822M) [application/zip]
Saving to: ‘glove.6B.zip.2’

glove

FileNotFoundError: ignored

In [ ]:
from absl import logging

import tensorflow as tf

import tensorflow_hub as hub
import matplotlib.pyplot as plt
import numpy as np
import os
import pandas as pd
import re
import seaborn as sns

module_url = "https://tfhub.dev/google/universal-sentence-encoder/4" #@param ["https://tfhub.dev/google/universal-sentence-encoder/4", "https://tfhub.dev/google/universal-sentence-encoder-large/5"]
model = hub.load(module_url)
print ("module %s loaded" % module_url)
def universe_embed(input):
  return model(input)

# Compute Metrics

**BLEU SCORES**: self-bleu and geometric mean-bleu




In [ ]:
a = [[1,2,3]]
b = [[2,4]]
a.extend(b)
a

[[1, 2, 3], [2, 4]]

In [ ]:
import nltk
nltk.download('punkt')
from nltk.translate.bleu_score import sentence_bleu


## self bleu 
def bleuScore(data): 
  # list of list of tokenized strings
  origs = []
  gens = []

  for batch in range(len(data): 
  # for batch in range(2*len(data): 
    #to do: (call original line) from data and append each to origs
    #to do: (call generated line) from data and append each to gens 
    
    score1 = corpus_bleu(origs, gens, weights=(0,0,0,0))
    score2 = corpus_bleu(origs, gens, weights=(0.5, 0.5, 0, 0))
    score3 = corpus_bleu(origs, gens, weights=(0.33, 0.33, 0.33, 0))
    score4 = corpus_bleu(origs, gens, weights=(0.25, 0.25, 0.25, 0.25))
    
  bleu_score = np.cumsum((score1*score2*score3*score4)**0.25)
  bleu_score = bleu_score / len(data)
  # bleu_score = bleu_score / (2*len(data))

  return bleu_score*100


**BERT SCORE:** between source and transferred embeddings

In [ ]:
pip install bert-score

In [ ]:
## bert score

from bert_score import score, BERTScorer

# def bertScore(originals, generated): #text sentences
#   scorer = BERTScorer(lang="en", rescale_with_baseline=True)
#   P, R, F1 = scorer.score(generated, originals)

# return F1

def bertScore(data): #embeddings
  origs = []
  gens= []
  for batch in range(len(data): 
  # for batch in range(2*len(data): 
    #to do: (call original line) from data
    origs.append()
    #to do: (call generated line) from data
    gens.append()
  _, _, F1 = score(gens, origs, lang="en", rescale_with_baseline=True)
  
  bertScore = np.cumsum(list(F1.numpy()))/len(list(F1.numpy()))*100

  return bertScore 




**Cosine Similarity**: between source and transferred embeddings
Embedding Options:
1. tf-idf (may be not a good choice because we don't actually have files but only sentences?)
2. GloVe
3. our trained encoder
4. Universal Sentence Encoder (https://colab.research.google.com/github/tensorflow/hub/blob/master/examples/colab/semantic_similarity_with_tf_hub_universal_encoder.ipynb)

In [ ]:
def cosine(data):
  similarities = []
  for batch in range(len(data): 
  # for batch in range(2*len(data): 
    #to do: original = (call each original line) from data ; souce embedding
    #to do: generated = (call each generated line) from data ; transferred embedding
    similarities.append(torch.cosine_similarity(original, generated).item())

  avg_cosine_similarity = np.sum(similarities)/len(data)
  # avg_cosine_similarity = np.sum(similarities)/(2*len(data))

  return avg_cosine_similarity



## **Fluency:**
general perplexity, data perplexity, and total perplexity

Relevant article:
https://www.scribendi.ai/comparing-bert-and-gpt-2-as-language-models-to-score-the-grammatical-correctness-of-a-sentence/

In [7]:
def id2word(self,indexes):
  """Convert a list of word indexes to string. Prepare for perplexity.
  Input: [int]
  Return: [str]"""
  result = []
  for i in indexes:
    result.append(self.id2v[i])
  return result

### Self model

In [8]:
def perplexity(probs):
  """probs: (batch, max_len, vocab_size) the probability of each word in each 
  position of the sentence. 
    It is also called logits_vectors as the output of decoder"""
  ppl = [0]*probs.shape[0]
  sentence_len = probs.shape[1]
  probs = torch.max(probs, dim = -1)
  for i in range(probs.shape[0]): 
    ppl[i] = torch.exp(torch.mean(torch.log(probs[i,:])))
  return torch.mean(ppl)


### Bert - Perplexity
Colab link: https://colab.research.google.com/github/huggingface/notebooks/blob/master/transformers_doc/quicktour.ipynb
Documentation link: https://huggingface.co/transformers/model_doc/bert.html#bertformaskedlm

In [10]:
! pip install transformers datasets
import torch
from transformers import BertTokenizer,BertForMaskedLM
# Load pre-trained model (weights)
with torch.no_grad():
    model = BertForMaskedLM.from_pretrained('bert-large-cased')
    model.eval()
    # Load pre-trained model tokenizer (vocabulary)
    tokenizer = BertTokenizer.from_pretrained('bert-large-cased')
def bert_ppl(sentence):
    tokenize_input = tokenizer.tokenize(sentence)
    tokenize_input = ["[CLS]"]+tokenize_input+["[SEP]"]
    tensor_input = torch.tensor([tokenizer.convert_tokens_to_ids(tokenize_input)])
    with torch.no_grad():
        loss=model(tensor_input, masked_lm_labels=tensor_input)[0]
    return torch.exp(loss.detach().numpy())

Downloading:   0%|          | 0.00/762 [00:00<?, ?B/s]

Downloading:   0%|          | 0.00/1.25G [00:00<?, ?B/s]

Some weights of the model checkpoint at bert-large-cased were not used when initializing BertForMaskedLM: ['cls.seq_relationship.bias', 'cls.seq_relationship.weight']
- This IS expected if you are initializing BertForMaskedLM from the checkpoint of a model trained on another task or with another architecture (e.g. initializing a BertForSequenceClassification model from a BertForPreTraining model).
- This IS NOT expected if you are initializing BertForMaskedLM from the checkpoint of a model that you expect to be exactly identical (initializing a BertForSequenceClassification model from a BertForSequenceClassification model).


Downloading:   0%|          | 0.00/208k [00:00<?, ?B/s]

Downloading:   0%|          | 0.00/29.0 [00:00<?, ?B/s]

Downloading:   0%|          | 0.00/426k [00:00<?, ?B/s]

## Pretrained GPT & GPT2 - Perplexity
pretrained GPT : https://huggingface.co/transformers/model_doc/gpt.html#openaigptlmheadmodel

GPT2 perplexity with fixed length instruction: https://huggingface.co/transformers/perplexity.html

In [11]:
from transformers import GPT2LMHeadModel, GPT2TokenizerFast, OpenAIGPTTokenizer, OpenAIGPTLMHeadModel
# GPT2
device = 'cuda'
model_id = 'gpt2-large'
GPT2model = GPT2LMHeadModel.from_pretrained(model_id).to(device)
GPT2tokenizer = GPT2TokenizerFast.from_pretrained(model_id)
# GPT
GPTmodel = OpenAIGPTLMHeadModel.from_pretrained('openai-gpt')
GPTtokenizer = OpenAIGPTTokenizer.from_pretrained('openai-gpt')


def gpt_ppl(transfered_words, gram_size = 1, 
            tokenizer= GPT2tokenizer, model = GPT2model):
  """
  Calculate the mean perplexity score of sentences returned 
  by the style transfer network. 
  input:
  - `transfered_sents`: a 2d-tensor of shape
      (batch_size, max_len) representing output sentence and
      the corresponding words 
  - `gram_size`: the sliding window size to calculate perplexity score. 
    this is same as the stride in the instruction.
  - `tokenizer`: can be either GPTtokenizer or GPT2tokenizer
      option 1 (GPT2): GPT2TokenizerFast.from_pretrained('gpt2-large')
      option 2 (GPT) : OpenAIGPTTokenizer.from_pretrained('openai-gpt')
  - `model`: which GPT model to generate probability
      option 1 (GPT2): GPT2LMHeadModel.from_pretrained('gpt2-large').to('cuda')
      option 2 (GPT) : OpenAIGPTLMHeadModel.from_pretrained('openai-gpt')
  return:
    - `ppls`: a list of perplexity scores for all sentences
  """
  gpt_max_length = model.config.n_positions 
  ppls = []
  for sent_i in range(transfered_words.shape[0]):

    encodings = GPT2tokenizer(' '.join(transfered_words[sent_i]), return_tensors='pt')
    nlls = []
    for i in tqdm(range(0, encodings.input_ids.size(1), gram_size)):
      begin_loc = max(i + gram_size - gpt_max_length, 0)
      end_loc = min(i + gram_size, encodings.input_ids.size(1))
      trg_len = end_loc - i    # may be different from gram_size on last loop
      input_ids = encodings.input_ids[:,begin_loc:end_loc].to(device)
      target_ids = input_ids.clone()
      target_ids[:,:-trg_len] = -100

      with torch.no_grad():
          outputs = model(input_ids, labels=target_ids)
          neg_log_likelihood = outputs[0] * trg_len

      nlls.append(neg_log_likelihood)

    ppl = torch.exp(torch.stack(nlls).sum() / end_loc)
    ppls.append(ppl)
  return ppls

Downloading:   0%|          | 0.00/666 [00:00<?, ?B/s]

Downloading:   0%|          | 0.00/3.02G [00:00<?, ?B/s]

Downloading:   0%|          | 0.00/0.99M [00:00<?, ?B/s]

Downloading:   0%|          | 0.00/446k [00:00<?, ?B/s]

Downloading:   0%|          | 0.00/1.29M [00:00<?, ?B/s]

...

Some weights of OpenAIGPTLMHeadModel were not initialized from the model checkpoint at openai-gpt and are newly initialized: ['lm_head.weight']
You should probably TRAIN this model on a down-stream task to be able to use it for predictions and inference.


Downloading:   0%|          | 0.00/797k [00:00<?, ?B/s]

Downloading:   0%|          | 0.00/448k [00:00<?, ?B/s]

Downloading:   0%|          | 0.00/1.21M [00:00<?, ?B/s]

ftfy or spacy is not installed using BERT BasicTokenizer instead of SpaCy & ftfy.


In [12]:
def tppl(gpt1_ppls, gpt2_ppls):
  return np.sqrt(torch.mean(gpt1_ppls)*torch.mean(gpt2_ppls))